In [1]:
%matplotlib inline
import tensorflow as tf
import matplotlib as plt
import glob
import numpy as np
from PIL import Image
import mltools as ml

trying to use the example from 
[http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/](http://warmspringwinds.github.io/tensorflow/tf-slim/2016/12/21/tfrecords-guide/)

In [2]:
labels ={"mesa":0, "forest":1}

In [ ]:
str([[1,0]]*len(files_rgb))

In [25]:
label = "mesa"
files_rgb = sorted(glob.glob( "/notebooks/Minecraft-AI/mc-data/"+label+"/"+label+"_rgb/*.jpg"), key=lambda x:x.split("_")[-1])
files_d = sorted(glob.glob("/notebooks/Minecraft-AI/mc-data/"+label+"/"+label+"_d/*.jpg"),key=lambda x:x.split("_")[-2])
filename_pairs = np.array(zip(files_rgb, files_d, [labels[label]]*len(files_rgb)))
label= "forest"
files_rgb = sorted(glob.glob( "/notebooks/Minecraft-AI/mc-data/"+label+"/"+label+"_rgb/*.jpg"), key=lambda x:x.split("_")[-1])
files_d = sorted(glob.glob("/notebooks/Minecraft-AI/mc-data/"+label+"/"+label+"_d/*.jpg"),key=lambda x:x.split("_")[-2])
filename_pairs = np.concatenate((filename_pairs, np.array(zip(files_rgb, files_d, [labels[label]]*len(files_rgb))))
                                , axis=0)
np.random.shuffle(filename_pairs)
portion=int(0.7*len(filename_pairs))
train = filename_pairs[:portion]
test = filename_pairs[portion:]
print(filename_pairs[0])

['/notebooks/Minecraft-AI/mc-data/forest/forest_rgb/forest_170.jpg'
 '/notebooks/Minecraft-AI/mc-data/forest/forest_d/forest_170_d.jpg' '1']


In [26]:
tfrecords_filename = '/notebooks/Minecraft-AI/mc-data/mesa_3_vs_forest_1_train.tfrecords'
tfrecords_test_filename='/notebooks/Minecraft-AI/mc-data/mesa_3_vs_forest_1_test.tfrecords'
train_writer = tf.python_io.TFRecordWriter(tfrecords_filename)
test_writer = tf.python_io.TFRecordWriter(tfrecords_test_filename)

In [27]:
# helper functions
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [28]:
# Let's collect the real images to later on compare
# to the reconstructed ones
original_images = []

In [29]:
def write_in_tfrecord(filename_pairs, writer):
    for img_path, annotation_path, label in filename_pairs:

        img = np.array(Image.open(img_path))
        # jenny zeng modified for ground truth
        annotation = (255 - np.array(Image.open(annotation_path)))
        
        # The reason to store image sizes was demonstrated
        # in the previous example -- we have to know sizes
        # of images to later read raw serialized string,
        # convert to 1d array and convert to respective
        # shape that image used to have.
        height = img.shape[0]
        width = img.shape[1]

        # Put in the original images into array
        # Just for future check for correctness
        original_images.append((img, annotation,label))

        img_raw = img.tostring()
        annotation_raw = annotation.tostring()

        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(height),
            'width': _int64_feature(width),
            'image_raw': _bytes_feature(img_raw),
            'mask_raw': _bytes_feature(annotation_raw),
            'label': _int64_feature(int(label))}))

        writer.write(example.SerializeToString())

In [30]:
write_in_tfrecord(train,train_writer)

In [31]:
write_in_tfrecord(test,test_writer)

In [32]:
train_writer.close()
test_writer.close()

In [57]:
reconstructed_images = []

In [58]:
record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_test_filename)

for string_record in record_iterator:
    
    example = tf.train.Example()
    example.ParseFromString(string_record)
    
    height = int(example.features.feature['height']
                                 .int64_list
                                 .value[0])
    
    width = int(example.features.feature['width']
                                .int64_list
                                .value[0])
    
    img_string = (example.features.feature['image_raw']
                                  .bytes_list
                                  .value[0])
    
    annotation_string = (example.features.feature['mask_raw']
                                .bytes_list
                                .value[0])
    label = int(example.features.feature['label']
                            .int64_list
                            .value[0])
    
    img_1d = np.fromstring(img_string, dtype=np.uint8)
    reconstructed_img = img_1d.reshape((height, width, -1))
    
    annotation_1d = np.fromstring(annotation_string, dtype=np.uint8)
    
    # Annotations don't have depth (3rd dimension)
    reconstructed_annotation = annotation_1d.reshape((height, width))
    
    reconstructed_images.append((reconstructed_img, reconstructed_annotation, label))

In [59]:
print(len(zip(original_images[0],reconstructed_images[0])))
count=0
print(zip(original_images[0],reconstructed_images[0])[2][1])

3
1


In [60]:
# # Let's check if the reconstructed images match
# # the original images
for i in range(len(original_images)):
    img_pair_to_compare, annotation_pair_to_compare, label_pair_to_compare = zip(original_images[i], reconstructed_images[i])

# for original_pair, reconstructed_pair, label_pair in zip(original_images, reconstructed_images):
#     img_pair_to_compare, annotation_pair_to_compare, label_pair_to_compare = zip(original_pair,
#                                                           reconstructed_pair, label_pair)
    print(np.allclose(*img_pair_to_compare))
    print(np.allclose(*annotation_pair_to_compare))
    print(np.allclose(*label_pair_to_compare))

False
False


TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''